In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

 I've written below code which can apply for Keras.
 FYR.
 If mistakes or improvement points are found, please feel free to post commnets!


 とりあえずKerasに放り込むところまで作りました。
 よければ参考にして下さい。間違いや改善点があればコメントで教えてほしいです。

Reading CSV and organizing a dataframe and PATHs.


CSVの読み込みとデータフレームPATH周りの調整を行っています。

In [ ]:
BASE_PATH = "/kaggle/input/happy-whale-and-dolphin/train_images/"
def addpath(x):
    return BASE_PATH + '/' + x
df = pd.read_csv("/kaggle/input/happy-whale-and-dolphin/train.csv")
df['image'] = df['image'].map(addpath)
#テストためデータフレームを適当な大きさに
#df_cut = df[0:100]
df_cut =　df

Loading the images datas and turn to numpy array. And then, augment the images.


画像の読み込みとnumpy行列化画像データの水増しを行っています。

In [ ]:
import numpy as np
from PIL import Image
import glob
IM_SIZE = 128
dummy = []
for index, i in enumerate(df_cut['image']):
    im = Image.open(i)
    im = im.resize((IM_SIZE, IM_SIZE))
    im = im.convert('L')
    np_im = np.array(im)
    df_cut['image'][index] = np_im
    #画像反転
    np_im_hr = np_im[:, ::-1]
    dummy.append([np_im_hr,df_cut['species'][index],df_cut['individual_id'][index]])
    np_im_ve = np_im[::-1,:]
    dummy.append([np_im_ve,df_cut['species'][index],df_cut['individual_id'][index]])
    #画像回転
    angle = [30,60,90,120,150,210,240,270,300,330]
    for ag in angle:
      rot = im.rotate(ag)
      rot = rot.resize((IM_SIZE,IM_SIZE))
      np_rot = np.array(rot)
      print(np_rot.shape)
      #np_rot = imresize(np_rot, (IM_SIZE, IM_SIZE))
      dummy.append([np_rot,df_cut['species'][index],df_cut['individual_id'][index]])
    print("\r"+str(index),end="")
#水増し分を追加
df_mizu = pd.DataFrame(dummy,columns = ['image','species','individual_id'])
df_cut = pd.concat([df_cut,df_mizu])
y = pd.get_dummies(df_cut['individual_id'])


A variable of df_mizu is a dataframe which includes numpy images which generated by flipping and rotating the original images.


df_mizuは水増ししたデータnumpy行列が格納されています。

In [ ]:
df_mizu = pd.DataFrame(dummy,columns = ['image','species','individual_id'])
df_cut = pd.concat([df_cut,df_mizu])
y = pd.get_dummies(df_cut['individual_id'])
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_cut['image'], y, test_size=0.1)
dummy = []
for i in x_train:
    flat = np.reshape(i,IM_SIZE,IM_SIZE)
    dummy.append(flat)
x_train0 = np.array(dummy)
dummy = []
for i in x_test:
    flat = np.reshape(i,IM_SIZE,IM_SIZE)
    dummy.append(flat)
x_test0 = np.array(dummy)
y_train0 = y_train.to_numpy()
y_test0 = y_test.to_numpy()

Learning by NN.


keras(NN)で学習。

In [ ]:
import tensorflow as tf
x_train0, x_test0 = x_train0 / 255.0, x_test0 / 255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(IM_SIZE,IM_SIZE)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(15587, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train0, y_train0, epochs=5, validation_data=(x_test0,y_test0))